In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets

import numpy as np

import os
import cv2

# Import required libraries
import pandas as pd
import numpy as np 
from numpy import vstack
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, mean_squared_error

from math import sqrt

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 

from datetime import datetime


from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn import MSELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import torch

In [30]:
# Hyperparameters
num_epochs = 6
num_classes = 30
batch_size = 100
learning_rate = 0.001

In [25]:
class wingsDataset(Dataset):
    
    def __init__(self, path):

        Xl = []
        yl = []
        
        for p in os.listdir(path):
            # categoria da imagem é definida pelo nome do arquivo
            category = p.split(" ")[0]
        
            # Abre a imagem usando opencv em escala de cinza
            img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)

            # Redimensionamento para 28 x 28 pixels
            new_img_array = cv2.resize(img_array, dsize=(28, 28))
        
            Xl.append(new_img_array)
            yl.append(category)
            #print(p)
        le = LabelEncoder()
        yll= le.fit_transform(yl)
        
        #yll=yll.astype(double)
        #print(yll)
        Xl=np.array(Xl)
        Xl = np.array(Xl).reshape(-1, 28,28,1)
        
        self.X = Tensor(Xl)
        self.y = Tensor(np.array(yll))
        print(torch.unique(self.y))
        self.X = self.X.permute(0, 3, 1, 2) # from NHWC to NCHW
 
    # quantas linhas tem no dataset?
    def __len__(self):
        return len(self.X)
 
    # obtem uma linha do dataset
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # retorna base para treino e teste
    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

path = '/home/silvio/dataset/bees/wingsEval/'
#path = '/home/silvio/dataset/bees/wingsEvalT/'
# Carrega Dataset   
dataset = wingsDataset(path)
# realiza split
train, test = dataset.get_splits()
   
# monta data loaders
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False)

tensor([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
        14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24.])


In [26]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 7 * 64, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [31]:
model = ConvNet()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
print(total_step)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        print(images.shape, labels.shape)
        labels = labels.to(torch.long)

        outputs = model(images)
        #print(outputs)
        print(labels)
        print(torch.unique(labels))
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))




12
torch.Size([100, 1, 28, 28]) torch.Size([100])
tensor([ 4, 18,  7,  2, 17,  4, 13,  5, 16, 21, 13, 23, 16, 24, 17, 11, 13, 12,
        17, 18,  3, 10, 18, 18,  0,  1, 17,  3, 16, 24,  2,  4, 10, 18,  9, 17,
        24, 21,  6, 11,  7, 23, 17, 21, 12,  4,  1, 11,  4, 12,  4,  4,  4, 10,
         4,  5, 17, 13, 21, 17, 10,  9,  4, 17,  9, 24,  3, 24, 21, 18,  0, 10,
        21, 10, 17,  3, 12, 15, 12,  9, 21, 17, 21, 21, 10,  8,  5, 17,  5, 12,
         2, 10, 16, 12, 21,  7, 13, 10, 16, 10])
tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
        21, 23, 24])


IndexError: Target 18 is out of bounds.

In [ ]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format((correct / total) * 100))

# Save the model and plot
#torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')

In [ ]:
from matplotlib import pyplot
pyplot.plot(loss_list, label='train')
#pyplot.plot(acc_list, label='test')
pyplot.legend()
pyplot.show()

In [ ]:
#pyplot.plot(loss_list, label='train')
pyplot.plot(acc_list, label='test')
pyplot.legend()
pyplot.show()